In [24]:
from datascience import *
import numpy as np
path_data = '../../../assets/data/'
np.set_printoptions(threshold=50)

# Exemplo: Tendências Populacionais

Agora estamos prontos para trabalhar com grandes tabelas de dados. O arquivo abaixo contém "Estimativas anuais da população residente por ano único de idade e sexo para os Estados Unidos." Observe que `read_table` pode ler dados diretamente de um URL.

In [25]:
# A partir de agosto de 2021, este arquivo do censo está online aqui: 
data = 'http://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/nc-est2019-agesex-res.csv'

# Uma cópia local pode ser acessada aqui caso o census.gov mova o arquivo: 
# data = path_data + 'nc-est2019-agesex-res.csv'

full_census_table = Table.read_table(data)
full_census_table

SEX,AGE,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019
0,0,3944153,3944160,3951430,3963092,3926570,3931258,3954787,3983981,3954773,3893990,3815343,3783052
0,1,3978070,3978090,3957730,3966225,3977549,3942698,3948891,3973133,4002903,3972711,3908830,3829599
0,2,4096929,4096939,4090621,3970654,3978925,3991740,3958711,3966321,3991349,4020045,3987032,3922044
0,3,4119040,4119051,4111688,4101644,3981531,3991017,4005928,3974351,3982984,4006946,4033038,3998665
0,4,4063170,4063186,4077346,4121488,4111490,3992502,4004032,4020292,3989750,3997280,4018719,4043323
0,5,4056858,4056872,4064521,4087054,4131049,4121876,4004576,4017589,4035033,4003452,4008443,4028281
0,6,4066381,4066412,4072904,4074531,4096631,4141126,4133372,4017388,4031568,4048018,4014057,4017227
0,7,4030579,4030594,4042990,4082821,4084175,4106756,4152666,4145872,4030888,4044139,4058370,4022319
0,8,4046486,4046497,4025501,4052773,4092559,4094513,4118349,4165033,4158848,4042924,4054236,4066194
0,9,4148353,4148369,4125312,4035319,4062726,4103052,4106068,4130887,4177895,4170813,4053179,4061874


Apenas as 10 primeiras linhas da tabela são exibidas. Mais tarde, veremos como exibir a tabela inteira; no entanto, isso geralmente não é útil com tabelas grandes.

Uma [descrição da tabela](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/nc-est2019-agesex-res.pdf) está disponível online. 

A coluna `SEX` contém códigos numéricos: `0` representa o total, `1` para masculino e `2` para feminino. As suposições subjacentes a essa categorização binária são questionáveis. Discutiremos isso na próxima seção.

Nesta seção, nos concentraremos na idade e no tamanho da população.

A coluna `AGE` contém idades em anos completos, mas o valor especial `999` é a soma da população total. A "idade" `100` também tem um status especial. Nessa categoria, o Census Bureau inclui todas as pessoas com 100 anos ou mais.

As outras colunas contêm estimativas da população dos EUA em cada categoria de sexo e idade nos anos de 2010 a 2019. O Censo é decenal: ocorre a cada 10 anos. O Censo mais recente foi realizado em 2020 e o anterior em 2010. O Census Bureau também estima mudanças na população a cada ano. Conforme explicado na descrição da metodologia do Bureau [aqui](https://www2.census.gov/programs-surveys/popest/technical-documentation/methodology/2010-2020/methods-statement-v2020-final.pdf), ele "adiciona [as mudanças estimadas] ao último censo decenal para produzir estimativas de população atualizadas a cada ano."

Tipicamente, uma tabela de acesso público conterá mais informações do que o necessário para uma investigação ou análise específica. Para tornar a tabela grande mais utilizável, precisamos fazer alguma *limpeza de dados*.

Suponha que estamos interessados apenas nas mudanças populacionais de 2014 a 2019. Vamos `select` as colunas relevantes.

In [26]:
partial_census_table = full_census_table.select('SEX', 'AGE', 'POPESTIMATE2014', 'POPESTIMATE2019')
partial_census_table

SEX,AGE,POPESTIMATE2014,POPESTIMATE2019
0,0,3954787,3783052
0,1,3948891,3829599
0,2,3958711,3922044
0,3,4005928,3998665
0,4,4004032,4043323
0,5,4004576,4028281
0,6,4133372,4017227
0,7,4152666,4022319
0,8,4118349,4066194
0,9,4106068,4061874


Podemos simplificar os rótulos das colunas selecionadas.

In [27]:
us_pop = partial_census_table.relabeled('POPESTIMATE2014', '2014').relabeled('POPESTIMATE2019', '2019')
us_pop

SEX,AGE,2014,2019
0,0,3954787,3783052
0,1,3948891,3829599
0,2,3958711,3922044
0,3,4005928,3998665
0,4,4004032,4043323
0,5,4004576,4028281
0,6,4133372,4017227
0,7,4152666,4022319
0,8,4118349,4066194
0,9,4106068,4061874


## Idades 97-100
Como aquecimento, vamos examinar a população total, rotulada pelo código `SEX` 0. Como todas essas linhas terão o mesmo valor 0 na coluna `SEX`, eliminaremos essa coluna.

In [28]:
us_pop_by_age = us_pop.where('SEX', are.equal_to(0)).drop('SEX')
us_pop_by_age

AGE,2014,2019
0,3954787,3783052
1,3948891,3829599
2,3958711,3922044
3,4005928,3998665
4,4004032,4043323
5,4004576,4028281
6,4133372,4017227
7,4152666,4022319
8,4118349,4066194
9,4106068,4061874


Agora vamos olhar para a população nas idades mais avançadas.

In [29]:
us_pop_by_age.where('AGE', are.between(97, 101))

AGE,2014,2019
97,82948,116969
98,59546,86150
99,41277,57124
100,70685,100322


Como era de se esperar, o número de pessoas é menor em idades mais avançadas. Por exemplo, há menos pessoas de 99 anos do que de 98 anos.

Mas os números para a idade `AGE` 100 são bastante maiores do que aqueles para a idade 99. Isso ocorre porque a linha com `AGE` 100 não representa apenas pessoas de 100 anos. Também inclui aquelas que têm mais de 100 anos. 

## Mudança Percentual

Cada coluna da tabela `us_pop_by_age` é uma matriz do mesmo comprimento, e assim as colunas podem ser combinadas usando aritmética. A matriz abaixo contém a mudança na população entre 2014 e 2019. Há uma entrada correspondente a cada linha de `us_pop_by_age`.

In [30]:
change = us_pop_by_age.column('2019') - us_pop_by_age.column('2014')
change

array([-171735, -119292,  -36667, ...,   15847,   29637, 9938515])

Podemos aumentar `us_pop_by_age` com uma coluna que contenha essas mudanças, tanto em termos absolutos quanto como porcentagens relativas ao valor em 2014.

In [31]:
us_pop_change = us_pop_by_age.with_columns(
    'Change', change,
    'Percent Change', change/us_pop_by_age.column('2014')
)
us_pop_change.set_format('Percent Change', PercentFormatter)

AGE,2014,2019,Change,Percent Change
0,3954787,3783052,-171735,-4.34%
1,3948891,3829599,-119292,-3.02%
2,3958711,3922044,-36667,-0.93%
3,4005928,3998665,-7263,-0.18%
4,4004032,4043323,39291,0.98%
5,4004576,4028281,23705,0.59%
6,4133372,4017227,-116145,-2.81%
7,4152666,4022319,-130347,-3.14%
8,4118349,4066194,-52155,-1.27%
9,4106068,4061874,-44194,-1.08%


Almost all the entries displayed in the `Percent Change` column are negative, demonstrating a drop in population at the youngest ages. However, the overall population grew by about 9.9 million people, a percent change of just over 3%.

In [32]:
us_pop_change.where('AGE', are.equal_to(999))

AGE,2014,2019,Change,Percent Change
999,318301008,328239523,9938515,3.12%


Vamos comparar isso com a mudança em cada idade. Para facilitar a interpretação, ordenaremos a tabela em ordem decrescente da mudança absoluta na população, contida na coluna `CHANGE`.

In [33]:
us_pop_change.where(
    'AGE', are.below(999)
).sort('Change', descending=True)

AGE,2014,2019,Change,Percent Change
72,2191642,3191048,999406,45.60%
68,2567511,3345475,777964,30.30%
69,2530460,3252423,721963,28.53%
70,2461426,3136704,675278,27.43%
71,2516392,3083083,566691,22.52%
76,1692960,2222392,529432,31.27%
62,3677408,4156645,479237,13.03%
28,4345247,4818725,473478,10.90%
64,3481789,3950578,468789,13.46%
38,3848856,4305576,456720,11.87%


Dê uma olhada nas primeiras linhas. Enquanto a mudança percentual é cerca de 3% para a população geral, ela salta para bem mais de 20% para as pessoas em seus finais dos sessenta e início dos setenta anos. Essa mudança impressionante contribui para o que é conhecido como o envelhecimento da América.

O que poderia explicar esse grande aumento? Podemos explorar essa questão examinando os anos em que os grupos relevantes nasceram.

- Aqueles que estavam na faixa etária de 69 a 72 anos em 2014 nasceram nos anos de 1942 a 1945. O ataque a Pearl Harbor ocorreu no final de 1941 e, em 1942, as forças dos EUA estavam fortemente envolvidas em uma guerra massiva que terminou em 1945.

- Aqueles que tinham de 69 a 72 anos em 2019 nasceram nos anos de 1947 a 1950, no auge do baby boom do pós-Segunda Guerra Mundial nos Estados Unidos.

O salto pós-guerra nos nascimentos é uma das principais razões para as grandes mudanças que observamos.